In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib import cm

from PIL import Image

<div style="text-align: center;">
    <span style="color: red; font-size: 80px;"> Esperienza 1 </span>
</div>

<div style="text-align: center;">
    <span style="color: blue; font-size: 60px;"> Calibrazione </span>
</div>

<div style="text-align: center;">
    <span style="color: grey; font-size: 45px;"> Studio delle immagini </span>
</div>

Le istantanee di plasma prese in fase di calibrazione sono 32: avendole aperte tutte posso affermare con certezza che nessuna delle immagini è corrotta. Procedo ad effettuare lo studio delle immagini considerandole tutte.

In [42]:
col = 1224
row = 1024

Le funzioni che verranno utilizzate per la pulizia sono le stesse che utilizzavo nel notebook dell'esperienza zero, ossia quella di calibrazione della trappola di Malberg-Penning ElTrap.

In [43]:
# Function to compute mean noise
def mean_noise(base, num_im):
    appo = 0
    m_noise = 0

    for i in range(1, num_im + 1):
        path = base + "{:03d}".format(i) + ".tif"
        m_noise = m_noise * i/(i+1) + np.array(Image.open(path))/(i+1)

    del appo
    return m_noise


# Function to set outside pixel to zero
def est_nullo(plasma, mean_noise, radius, xcen, ycen):
    x = np.linspace(-xcen, col - xcen, col)
    y = np.linspace(-ycen, row - ycen, row)

    # Creating mash grid to check wether we are in trap or not
    X, Y = np.meshgrid(x, y)
    
    plasma = plasma - mean_noise

    mask = plasma < 0
    plasma[mask] = 0

    mask = (X*X + Y*Y) > pow(radius, 2)
    plasma[mask] = 0

    del mask, x, y, X, Y
    return plasma


# Function to reduce additional noise
def no_rifl(plasma, r_pl, r_trap, xcen, ycen):
    x = np.linspace(-xcen, col - xcen, col)
    y = np.linspace(-ycen, row - ycen, row)

    X, Y = np.meshgrid(x, y)

    mask = ((X*X + Y*Y) < pow(r_trap, 2)) & ((X*X + Y*Y) > pow(r_pl, 2))
    appo = np.mean(plasma[mask])

    mask = (X*X + Y*Y) < pow(r_trap, 2)
    plasma[mask] = plasma[mask] - appo

    mask = plasma < 0
    plasma[mask] = 0
    
    del x, y, X, Y, appo, mask
    return plasma


<center>

### Stampa delle immagini pulite

</center>

In [47]:
rum_med = mean_noise("Dati/CAMimages/dark/dark", 80) 
xcen = 612; ycen = 502; r_pl = 320; rtrap = 419

conta = 1
for i in range(1, 33):

    path = "Dati/CAMimages/calibrazione/calibplasma" + "{:03d}".format(i) + ".tif"

    plasma = np.array(Image.open(path))
    plasma = plasma.reshape(1024, 1224)

    plasma = est_nullo(plasma, rum_med, rtrap, xcen, ycen)
    plasma = no_rifl(plasma, r_pl, rtrap, xcen, ycen)

    # Salvo l'immagine
    path = "Dati/CAMimages/plasma_pulite/plasma" + "{:03d}".format(conta) + ".tif"

    image = Image.fromarray(plasma)
    image.save(path)
    conta += 1

del rum_med, plasma, image, conta
del xcen, ycen, r_pl, rtrap, path

<center>

### Calcolo dell'intensità media

</center>

In [48]:
int_media = 0

for i in range(1, 33):
    path = "Dati/CAMimages/plasma_pulite/plasma" + "{:03d}".format(i) + ".tif"
    plasma = np.array(Image.open(path))
    plasma = plasma.reshape(1024, 1224)

    int_media = int_media * (i-1)/i + sum(sum(plasma))/i

del path, plasma
print("L'intensità media delle immagini di plasma è: ", int(int_media))

L'intensità media delle immagini di plasma è:  31620138


<div style="text-align: center;">
    <span style="color: grey; font-size: 45px;"> Studio della scarica </span>
</div>

In [50]:
def moving_av(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

def exp_func(x, a, b):
    return a * np.exp(b * x)

<center>

### Scarica ionica

</center>

I venti segnali di scarica ionica sono tutti simili fra loro e non presentano criticità. Per questo motivo per quantificare la carica positiva presente nella trappola, utilizzo tutti i venti segnali.

In [98]:
udm_v = 1e-3; udm_t = 1e-3
res = 1e6; car_ioni = []

for i in range(1, 21):

    #----------------------------------------#
    #          Lettura file di input         #
    #----------------------------------------#
    filename = 'Dati/Signals/calibration/idischarge/idischarge_' + "{:02d}".format(i) + '.txt'

    data = pd.read_csv(filename, delim_whitespace=True, skiprows=3, header=None)
    t = data[0].astype(float) * udm_t; v = data[1].astype(float) * udm_v


    #----------------------------------------#
    #      Calcolo offset e sottrazione      #
    #----------------------------------------#
    v_off = np.mean(v[:30000])
    v = v - v_off


    #----------------------------------------#
    #          Smoothing del segnale         #
    #----------------------------------------#      
    v_smooth = moving_av(v, window_size = 100)
    t = t[:len(v_smooth)]


    #----------------------------------------#
    #             Fit esponenziale           #
    #----------------------------------------#   
    t_fit = t[41000:60000]
    popt, pcov = curve_fit(exp_func, t_fit, v_smooth[41000:60000])
    a, b = popt


    #----------------------------------------#
    #          Calcolo della carica          #
    #----------------------------------------#
    mask = t > 0; dt = t[155] - t[154]; cap = -1.0/(b * res)
    car_ioni.append(np.sum(v_smooth[mask]) * dt/res + np.mean(v_smooth[len(v_smooth) - 101 : len(v_smooth) - 1]) * cap)

car_ioni = np.array(car_ioni)
print("La carica media della popolazione ionica risulta: " + str(round(np.mean(car_ioni), 13))+ r" +/- " + str(round(np.std(car_ioni), 13)) + " C")
del res, udm_v, udm_t, mask, dt, a, b, t_fit, t, v, v_smooth, cap

La carica media della popolazione ionica risulta: 1.35e-11 +/- 7e-13 C


<center>

### Scarica elettronica

</center>